In [1]:
import os
import hydra
import pytorch_lightning as pl
import torch
#from puzzlefusion_plusplus.auto_aggl import AutoAgglomerative
from omegaconf import OmegaConf,open_dict
import omegaconf



In [2]:
cfg_auto_aggl = omegaconf.OmegaConf.load('config/auto_aggl.yaml')
cfg_denoiser_data = omegaconf.OmegaConf.load('config/denoiser/data.yaml')
cfg_denoiser_encode = omegaconf.OmegaConf.load('config/denoiser/encoder.yaml')
cfg_denoiser_global_config = omegaconf.OmegaConf.load('config/denoiser/global_config.yaml')
cfg_denoiser_model = omegaconf.OmegaConf.load('config/denoiser/model.yaml')

cfg_ae_vq_vae = omegaconf.OmegaConf.load('config/ae/vq_vae.yaml')
cfg_ae_global_config = omegaconf.OmegaConf.load('config/ae/global_config.yaml')
cfg_ae_model = omegaconf.OmegaConf.load('config/ae/model.yaml')


cfg = OmegaConf.merge(
    cfg_auto_aggl,
    {"denoiser": cfg_denoiser_data},
    {"denoiser": cfg_denoiser_encode},
    {"denoiser": cfg_denoiser_global_config},
    {"denoiser": cfg_denoiser_model},
    {"ae": cfg_ae_vq_vae},
    {"ae": cfg_ae_global_config},
    {"ae": cfg_ae_model},
    #{"verifier": omegaconf.OmegaConf.load('config/verifier/global_config.yaml')},
    #{"verifier": omegaconf.OmegaConf.load('config/verifier/model.yaml')},
)

In [3]:

data_home_dir = '/data/jhahn/data/shape_dataset/'
data_type_name = 'shape'

with open_dict(cfg):
    cfg.experiment_output_path = data_home_dir+'experiment_output/'
    cfg.denoiser.data.data_val_dir=f'{data_home_dir}/pc_data/{data_type_name}/val/'
    #cfg.denoiser.ckpt_path= data_home_dir+f'output/denoiser/everyday_epoch100_bs64/training/last.ckpt'
    cfg.denoiser.ckpt_path= data_home_dir+f'output/denoiser/shape_epoch10/training/last.ckpt'
    cfg.inference_dir= data_home_dir+'results'
    cfg.denoiser.data.val_batch_size=1
    #cfg.verifier.ckpt_path= '/disk2/data/breaking-bad-dataset/output/verifier/everyday_epoch100_bs64/training/last.ckpt'

#print(OmegaConf.to_yaml(cfg))
#print(cfg.experiment_output_path)

In [4]:
# fix the seed
pl.seed_everything(cfg.test_seed, workers=True)

# create directories for inference outputs
inference_dir = os.path.join(cfg.experiment_output_path, "inference", cfg.inference_dir)
os.makedirs(inference_dir, exist_ok=True)

# initialize data
from jahn_src.data_loader_jhahn import build_test_dataloader


#from puzzlefusion_plusplus.denoiser.dataset.dataset import build_test_dataloader
#cfg.denoiser.data.matching_data_path= '/disk2/data/breaking-bad-dataset/data/matching_data'

test_loader = build_test_dataloader(cfg.denoiser)

Seed set to 123
100%|██████████| 10/10 [00:00<00:00, 973.79it/s]


In [24]:
import importlib
import jahn_src.auto_agg_jhahn
import jahn_src.verifier_jhahn
importlib.reload(jahn_src.verifier_jhahn)
importlib.reload(jahn_src.myutil)
importlib.reload(jahn_src.auto_agg_jhahn)
from jahn_src.auto_agg_jhahn import AutoAgglomerative

#import importlib
#import puzzlefusion_plusplus.auto_aggl
#importlib.reload(puzzlefusion_plusplus.auto_aggl)
#from puzzlefusion_plusplus.auto_aggl import AutoAgglomerative

In [25]:

model = AutoAgglomerative(cfg)

denoiser_weights = torch.load(cfg.denoiser.ckpt_path)['state_dict']

model.denoiser.load_state_dict(
    {k.replace('denoiser.', ''): v for k, v in denoiser_weights.items() 
     if k.startswith('denoiser.')}
)

model.encoder.load_state_dict(
    {k.replace('encoder.', ''): v for k, v in denoiser_weights.items() 
     if k.startswith('encoder.')}
)

# load verifier weights    
#verifier_weights = torch.load(cfg.verifier.ckpt_path)['state_dict']
#model.verifier.load_state_dict({k.replace('verifier.', ''): v for k, v in verifier_weights.items()})


trainer = pl.Trainer(accelerator=cfg.accelerator, max_epochs=1, logger=False)
trainer.test(model=model, dataloaders=test_loader)


TypeError: new(): data must be a sequence (got float)